In [1]:
import pandas as pd
import pickle
import os
import numpy as np

In [2]:
def load_data_one_stock(path):
 
  file = open(path, 'rb')
  load_data= pickle.load(file)
  file.close()

  return load_data
  

In [45]:
# Settings 
# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA']
stock_name_list  = ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']
folder_saved_models = f'historical_data'
parent_path = os.getcwd()
buy_money = 1000

In [4]:
## Load data

# file_name = 'df_stock_AAPL_period2y_interval1h.pkl'
# path = os.path.join(parent_path, folder_saved_models, file_name)
## stock_name = item[str.find(item, 'df_') + 3 : str.find(item, '_20')]
# df = load_data_one_stock(path)
## df.plot()
## print(df)

In [5]:
def is_near_global_max(i, k=400, prt=70):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   reference_point = df['close'].iloc[i - k]
  else:
   reference_point = df['close'].iloc[0]
   gmax = max(df['close'].iloc[0: i])
 
  if gmax == reference_point:
    result = False
  elif 100 * (df['close'].iloc[i] - reference_point) / (gmax - reference_point) > prt:
    result = True
  else:
    result = False

  return result

In [48]:
def one_stock_simulation(df, ticker, print_sim_results=False, print_trans_history=False):

  history_log = {'number success': 0,
                'number fails': 0,
                'profit': 0,
                'transactions': [] }  
  last_top = df['close'].iloc[0]
  last_top_time = df['close'].index[0]
  last_top_i = 0
  reverse_flag = False
  sell_finished = True

  for i in range(2, df.shape[0]):

    # dynaminc profit/loose coefficient
    if i > 200:
      if df['close'].iloc[i] / df['close'].iloc[i - 200] > 1.05: # bull trend
        gain_coef = 1.02
        lose_coef = 0.98  # not equal to 1 - k (DO Different Expertiments)
      else: # bear trend
        gain_coef = 1.005
        lose_coef = 0.95
    else: # undefined trend
      gain_coef = 1.02
      lose_coef = 0.98

    if sell_finished == False and df['high'].iloc[i] / buy_price > gain_coef:
      sell_price = float(df['high'].iloc[i])
      history_log['number success'] += 1
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks
      history_log['transactions'].append({
        'type': 'success',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2), 
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['high'].index[i]
     })
      sell_finished = True

    if sell_finished == False and df['low'].iloc[i] / buy_price < lose_coef :
      sell_price = float(df['low'].iloc[i])
      sell_finished = True
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks
      history_log['number fails'] += 1
      history_log['transactions'].append({
        'type': 'fail',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2),
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['low'].index[i]
      })

    # KEY parameters: RIV, prt, last_top_ratio
    # buy_time_cond = (df.index[i]).hour <= 15 and (df.index[i]).hour >= 11

    buy_ratio = float(last_top -  df['open'].iloc[i]) / float(df['ha_pct'].iloc[i])
    if df['ha_pct'].iloc[i] > RIV \
      and reverse_flag==False \
      and sell_finished == True \
      and last_top / df['open'].iloc[i] > last_top_ratio \
      and i - last_top_i > distance_from_last_top \
      and buy_time_cond \
      and buy_ratio > buy_ratio_border \
      and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)):

      reverse_flag = True
      buy_price = float(df['close'].iloc[i])
      buy_time = df.index[i]
      buy_ratio_hl = buy_ratio
      sell_finished = False

    # last top and reverse flag
    if df['ha_colour'].iloc[i] == 'red' \
      and df['ha_colour'].iloc[i - 1] == 'green'\
      and df['ha_colour'].iloc[i - 2] == 'green'\
      and df['ha_pct'].iloc[i - 1] > 0.1 \
      and df['ha_pct'].iloc[i - 2] > 0.1:
  
      last_top = df['high'].iloc[i - 1]
      last_top_i = i - 1
      reverse_flag = False
  
  # Simulation results 

  if print_sim_results:
    print('-'*50)
    print(f'Simulation results for ticker {ticker}:')
    print(history_log)
  
  if print_trans_history:
    for tran in history_log['transactions']:
      print(tran)

  return history_log['profit'], history_log['number fails']


In [12]:
buy_time_cond = True
buy_ratio_border = 7
distance_from_last_top  = 0  # NOT SIGNIFICANT?
last_top_ratio = 1.002 # last top / current open price  NOT SIGNIFICANT?
RIV = 0.1 # Reversal indication Value
is_near_global_max_prt = 96 # usd to be 96 

parameters_list_search = {
  'buy_ratio_border' : [0,3,5,7,9,10],
  'distance_from_last_top': [0,2,3,5],
  'last_top_ratio': [1, 1.001, 1.002, 1.003],
  'RIV' : [0.05, 0.1, 0.15, 0.2, 0.25],
  'is_near_global_max_prt' : [0, 20, 30, 50, 70, 80, 96, 100]
}

In [50]:
optimize_profit = False
for parameter in parameters_list_search:
  total_profit_list = []
  total_loses_list = []
  for value in parameters_list_search[parameter]:
    vars()[parameter] = value
    print(f'parametr is {parameter} value is {value}:')
    total_profit = 0
    total_loses = 0
    for ticker in stock_name_list:
      file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
      path = os.path.join(parent_path, folder_saved_models, file_name)
      df = load_data_one_stock(path)
      profit, loses= one_stock_simulation(df, ticker)
      total_profit += profit
      total_loses += loses
    total_profit_list.append(total_profit)
    total_loses_list.append(total_loses)
    print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}')
  print(total_profit_list)
  if optimize_profit:
    vars()[parameter] = parameters_list_search[parameter][np.argmax(total_profit_list)]
  else:
    vars()[parameter] = parameters_list_search[parameter][np.argmin(total_loses_list)]
  print(f'parameter {parameter} best value is {vars()[parameter]}')
  # np.max(total_profit_list), np.argmax(total_profit_list)

parameters_list_best = {}
for parameter in parameters_list_search:
  print(f'parameter {parameter} best value is {vars()[parameter]}')
  
  parameters_list_best[parameter] = vars()[parameter]
  
print(parameters_list_best)
print(max(total_profit_list), min(total_loses_list))


parametr is buy_ratio_border value is 0:
Total Profit is 2652.18, total loses is 130
parametr is buy_ratio_border value is 3:
Total Profit is 1996.07, total loses is 107
parametr is buy_ratio_border value is 5:
Total Profit is 1676.19, total loses is 91
parametr is buy_ratio_border value is 7:
Total Profit is 918.96, total loses is 77
parametr is buy_ratio_border value is 9:
Total Profit is 936.13, total loses is 62
parametr is buy_ratio_border value is 10:
Total Profit is 874.96, total loses is 53
[2652.1774711608887, 1996.0676612854004, 1676.1946487426758, 918.961311340332, 936.1323699951172, 874.9648590087891]
parameter buy_ratio_border best value is 10
parametr is distance_from_last_top value is 0:
Total Profit is 1064.06, total loses is 54
parametr is distance_from_last_top value is 2:
Total Profit is 1022.38, total loses is 54
parametr is distance_from_last_top value is 3:
Total Profit is 886.86, total loses is 54
parametr is distance_from_last_top value is 5:
Total Profit is 874

In [ ]:
{'buy_ratio_border': 9, 'distance_from_last_top': 5, 'last_top_ratio': 1, 'RIV': 0.1, 'is_near_global_max_prt': 96}
864.1497821807861 85

{'buy_ratio_border': 10, 'distance_from_last_top': 5, 'last_top_ratio': 1, 'RIV': 0.25, 'is_near_global_max_prt': 0}
466.66666412353516 46

In [ ]:
# 0.2 for both win and loses
# profit 2899 total loses 260
{'buy_ratio_border': 0, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.05, 'is_near_global_max_prt': 96}

# profit 966 total loses 13
{'buy_ratio_border': 10, 'distance_from_last_top': 5, 'last_top_ratio': 1.002, 'RIV': 0.25, 'is_near_global_max_prt': 0}

In [18]:
# 0.2 for both win and 0.5 for loses
#profit is 4242
best_profit_values = {'buy_ratio_border': 0, 'distance_from_last_top': 3, 'last_top_ratio': 1.001, 'RIV': 0.1, 'is_near_global_max_prt': 96}

#profit is 1580 loses is 38
{'buy_ratio_border': 10, 'distance_from_last_top': 5, 'last_top_ratio': 1, 'RIV': 0.25, 'is_near_global_max_prt': 0}


In [10]:
for parameter in parameters_list_search:
  print(f'parameter {parameter} best value is {vars()[parameter]}')

parameter buy_ratio_border best value is 0
parameter distance_from_last_top best value is 3
parameter last_top_ratio best value is 1
parameter RIV best value is 0.1
parameter is_near_global_max_prt best value is 96


In [51]:
# TRY TO REDUCE NUMBER OF FAIL NOT PROFIT!!!
total_profit_list = []
# parametr_list = np.arange(86, 100, 1)
# parametr_list = [1, 1.001, 1.002, 1.005, 1.007, 1.01, 1.02]
# parametr_list = [30]
# for p in parametr_list:
#   is_near_global_max_prt= p
 
#   print(f'parametr is {p}:')
total_profit = 0
total_loses = 0
for ticker in stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  profit, loses = one_stock_simulation(df, ticker, print_sim_results=True, print_trans_history=True)
  total_profit += profit
  total_loses += loses
# total_profit_list.append(total_profit)
print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}')
# np.max(total_profit_list), np.argmax(total_profit_list)

--------------------------------------------------
Simulation results for ticker GOOGL:
{'number success': 10, 'number fails': 5, 'profit': 42.29402160644531, 'transactions': [{'type': 'fail', 'buy_price': 110.5, 'sell_price': 107.18, 'ratio': 20.75, 'buy_time': Timestamp('2022-06-30 11:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-07-01 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 110.75, 'sell_price': 113.03, 'ratio': 26.51, 'buy_time': Timestamp('2022-07-14 13:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-07-15 09:30:00-0400', tz='America/New_York')}, {'type': 'fail', 'buy_price': 107.99, 'sell_price': 105.25, 'ratio': 11.24, 'buy_time': Timestamp('2022-07-25 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-07-26 09:30:00-0400', tz='America/New_York')}, {'type': 'fail', 'buy_price': 104.96, 'sell_price': 99.52, 'ratio': 12.16, 'buy_time': Timestamp('2022-09-14 12:30:00-0400', tz='America/New_Yor